In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pyautogui as pg
import time
from tqdm import tqdm
import pandas as pd
from config import url, username, password

df =  pd.read_csv('Portland Area Active Email.csv')
actives = df['Name'].values.tolist()
actives = [h.title() for h in actives]

url = url
response = requests.get(url)
chrome_options = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

username = driver.find_element( By.ID, 'user_email')
password = driver.find_element(By.ID, 'user_password')

# enter the login credentials
username.send_keys(username)
password.send_keys(password)

login_button = driver.find_element(By.NAME, 'commit')
login_button.click()

dropdown = driver.find_element(By.ID, 'user-usurp')
dropdown.click()

texts = ['Therapydia Portland', 'Therapydia Tanasbourne', 'Therapydia Lake Oswego', 'Therapydia Beaverton', 'Therapydia Pearl', 'Therapydia Vancouver', 'Therapydia N. Portland', 'Therapydia Progress Ridge', 'Therapydia Hazel Dell']

location = driver.find_element(By.LINK_TEXT, texts[0])
location.click()

sources = driver.find_element(By.LINK_TEXT, 'Sources')
sources.click()

for g in actives:
    search_referrer = driver.find_element(By.NAME, 'search_referrer')
    search_referrer.send_keys(g)
    pg.press('enter')
    time.sleep(1)
    try:
        checkbox = driver.find_element(By.XPATH, f"//input[@type='checkbox'][@class='n-email ref_check y-mailer']")
        checkbox.click()
        time.sleep(1)
        bulk_dropdown = driver.find_element(By.LINK_TEXT, 'Select Bulk Action')
        bulk_dropdown.click()

        time.sleep(.5)

        add_to_email = driver.find_element(By.ID, 'add_actives_to_email_list')
        add_to_email.click()

        time.sleep(1)

        list_added = pg.locateCenterOnScreen(r'C:\Users\Madias\Documents\Data\portland_email_dedupe\correct_list.png', confidence=.8)
        if list_added:
            finalize_list = pg.locateCenterOnScreen(r'C:\Users\Madias\Documents\Data\portland_email_dedupe\add.png',confidence=.8)
            pg.click(finalize_list[0],finalize_list[1])
        else:    
            add_overlay = pg.locateCenterOnScreen(r'C:\Users\Madias\Documents\Data\portland_email_dedupe\add_email.png',confidence=.8)
            pg.click(add_overlay[0]+50 ,add_overlay[1]+87 )
            time.sleep(2)
            choose_campaign = pg.locateCenterOnScreen(r'C:\Users\Madias\Documents\Data\portland_email_dedupe\email_pick.png',confidence=.8)
            pg.click(choose_campaign[0],choose_campaign[1])
            time.sleep(2)
            finalize_list = pg.locateCenterOnScreen(r'C:\Users\Madias\Documents\Data\portland_email_dedupe\add.png',confidence=.8)
            pg.click(finalize_list[0],finalize_list[1])
    except:   
        try: 
            checkbox = driver.find_element(By.XPATH, f"//input[@type='checkbox'][@class='ref_check y-email y-mailer']")
            checkbox.click()
            bulk_dropdown = driver.find_element(By.LINK_TEXT, 'Select Bulk Action')
            bulk_dropdown.click()

            time.sleep(.5)

            add_to_email = driver.find_element(By.ID, 'add_actives_to_email_list')
            add_to_email.click()

            time.sleep(1)

            list_added = pg.locateCenterOnScreen(r'C:\Users\Madias\Documents\Data\portland_email_dedupe\correct_list.png', confidence=.8)
            if list_added:
                finalize_list = pg.locateCenterOnScreen(r'C:\Users\Madias\Documents\Data\portland_email_dedupe\add.png',confidence=.8)
                pg.click(finalize_list[0],finalize_list[1])
            else:    
                add_overlay = pg.locateCenterOnScreen(r'C:\Users\Madias\Documents\Data\portland_email_dedupe\add_email.png',confidence=.8)
                pg.click(add_overlay[0]+50 ,add_overlay[1]+87 )
                time.sleep(2)
                choose_campaign = pg.locateCenterOnScreen(r'C:\Users\Madias\Documents\Data\portland_email_dedupe\email_pick.png',confidence=.8)
                pg.click(choose_campaign[0],choose_campaign[1])
                time.sleep(2)
                finalize_list = pg.locateCenterOnScreen(r'C:\Users\Madias\Documents\Data\portland_email_dedupe\add.png',confidence=.8)
                pg.click(finalize_list[0],finalize_list[1])
        except:
            pass

    time.sleep(1)
    actives.remove(g)

    search_referrer.clear()

